In [1]:
import httplib2
import argparse
import pandas as pd
import requests
import json
from pprint import pprint
from datetime import datetime, timedelta
from yaml import load, FullLoader
from pprint import pprint
from oauth2client.file import Storage
from oauth2client import tools
from oauth2client import client
from googleapiclient import discovery

In [2]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
CLIENT_SECRET_FILE = 'client_secret_grants.json' 
APPLICATION_NAME = 'Google Sheets API Report'
credential_path = 'sheets.googleapis.com-report.json'

In [3]:
store = Storage(credential_path)
credentials = store.get()
parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, parents=[tools.argparser])
flags = parser.parse_args([])
if not credentials or credentials.invalid:
    flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
    flow.user_agent = APPLICATION_NAME
    if flags:
        credentials = tools.run_flow(flow, store, flags)
print('Storing credentials to ' + credential_path)

Storing credentials to sheets.googleapis.com-report.json


In [4]:
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?version=v4')
service = discovery.build('sheets', 'v4', http = http, discoveryServiceUrl = discoveryUrl)
spreadsheetId = '1CmQqTVuzqDs5WX7fNAlB43AvTsoOfYqIE266impvCJI'

In [5]:
rangeName = 'partners dict!A1:C10'

In [6]:
result = service.spreadsheets().values().get(spreadsheetId = spreadsheetId, range = rangeName).execute()
values = result.get('values', [])

In [7]:
pprint(values)

[['Партнер', 'utm_source', 'utm_medium'],
 ['СМИ2', 'smi2', 'cpm'],
 ['РБК', 'rbc', 'cpc'],
 ['Infox', 'infox', 'cpc'],
 ['Infox', 'infox-action', 'cpa']]


In [8]:
headers = values[0]
rows = values[1:]
data_top = pd.DataFrame.from_records(rows, columns = headers)
data_bottom = pd.read_csv('data/metrika_data.txt', sep = '\t')

In [9]:
data_top.head()

,Партнер,utm_source,utm_medium
0,СМИ2,smi2,cpm
1,РБК,rbc,cpc
2,Infox,infox,cpc
3,Infox,infox-action,cpa


In [10]:
data_bottom.head()

,utm_source,utm_medium,sales
0,infox,cpc,1534
1,smi2,cpm,1495
2,infox-action,cpa,735
3,rbc,cpc,59


In [11]:
result = pd.merge(data_top, data_bottom, how='outer', on=['utm_source', 'utm_medium'])

In [12]:
result.head()

,Партнер,utm_source,utm_medium,sales
0,СМИ2,smi2,cpm,1495
1,РБК,rbc,cpc,59
2,Infox,infox,cpc,1534
3,Infox,infox-action,cpa,735


## Параметры Яндекс метрики

In [13]:
config_file = open('config.yaml', 'r')
config = load(config_file, Loader=FullLoader)
token = config['token']
headers = {'Authorization': 'OAuth ' + token}
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
counter = '51583145'
startDate = '2018-12-01'
endDate = '2020-12-31'
dimensions = ['ym:s:date', 'ym:s:deviceCategory', 'ym:s:hasAdBlocker', 'ym:s:trafficSource']
metrics = ['ym:s:visits', 'ym:s:users', 'ym:s:pageviews']
metrics_string = ','.join(metrics)
dimensions_string = ','.join(dimensions)

In [14]:
f = open('data/report_params.yaml', 'r')
params = load(f, Loader=FullLoader)
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
startDate = (datetime.now() - timedelta(days = params['reports']['dau']['days_ago'])).strftime('%Y-%m-%d')
endDate = (datetime.now() - timedelta(days = 1)).strftime('%Y-%m-%d')
dimensions = params['reports']['dau']['dimensions']
metrics = params['reports']['dau']['metrics']
sortby = params['reports']['dau']['sort']
counter = params['metrika counter']

In [15]:
def make_request(**kwargs):
    """Запрос к API Яндекс.Метрики. Возвращает JSON ответа"""
    return requests.get(API_URL, headers=headers, params = kwargs).json()

In [16]:
def json_handler(data):
    """Преобразование ответа JSON в спикок листов"""
    report = []
    for line in data['data']:
        dimension_list = [x['name'] for x in line['dimensions']]
        metrics_list = line['metrics']
        report.append(dimension_list + metrics_list)
    return report

In [17]:
response = make_request(date1 = startDate, date2 = endDate, dimensions = dimensions, metrics = metrics, 
                    id = counter, sort = sortby, accuracy = 1)

In [18]:
data = json_handler(response)
data

[['2020-11-27', 109.0, 98.0, 228.0, 25.68807339],
 ['2020-11-28', 85.0, 75.0, 162.0, 24.70588235],
 ['2020-11-29', 63.0, 58.0, 104.0, 28.57142857],
 ['2020-11-30', 84.0, 76.0, 178.0, 21.42857143],
 ['2020-12-01', 82.0, 77.0, 149.0, 29.26829268],
 ['2020-12-02', 74.0, 71.0, 128.0, 21.62162162],
 ['2020-12-03', 70.0, 67.0, 157.0, 25.71428571],
 ['2020-12-04', 81.0, 75.0, 149.0, 27.16049383],
 ['2020-12-05', 85.0, 79.0, 164.0, 22.35294118],
 ['2020-12-06', 66.0, 60.0, 124.0, 30.3030303],
 ['2020-12-07', 95.0, 89.0, 183.0, 22.10526316],
 ['2020-12-08', 74.0, 67.0, 139.0, 21.62162162],
 ['2020-12-09', 89.0, 82.0, 189.0, 21.34831461],
 ['2020-12-10', 81.0, 79.0, 154.0, 27.16049383],
 ['2020-12-11', 76.0, 70.0, 140.0, 25.0],
 ['2020-12-12', 70.0, 67.0, 134.0, 17.14285714],
 ['2020-12-13', 72.0, 70.0, 141.0, 18.05555556],
 ['2020-12-14', 65.0, 62.0, 134.0, 13.84615385],
 ['2020-12-15', 76.0, 70.0, 150.0, 17.10526316],
 ['2020-12-16', 93.0, 81.0, 181.0, 18.27956989],
 ['2020-12-17', 89.0, 86.0,

In [19]:
range_data = params['reports']['dau']['range_name']
range_name = '{}!{}{}:{}{}'.format(range_data['list'], range_data['start'], 2, range_data['end'], len(data) + 1)

In [20]:
range_name

'daily_metrics!A2:E31'

In [21]:
body = {'values': data}

In [22]:
pprint(body)

{'values': [['2020-11-27', 109.0, 98.0, 228.0, 25.68807339],
            ['2020-11-28', 85.0, 75.0, 162.0, 24.70588235],
            ['2020-11-29', 63.0, 58.0, 104.0, 28.57142857],
            ['2020-11-30', 84.0, 76.0, 178.0, 21.42857143],
            ['2020-12-01', 82.0, 77.0, 149.0, 29.26829268],
            ['2020-12-02', 74.0, 71.0, 128.0, 21.62162162],
            ['2020-12-03', 70.0, 67.0, 157.0, 25.71428571],
            ['2020-12-04', 81.0, 75.0, 149.0, 27.16049383],
            ['2020-12-05', 85.0, 79.0, 164.0, 22.35294118],
            ['2020-12-06', 66.0, 60.0, 124.0, 30.3030303],
            ['2020-12-07', 95.0, 89.0, 183.0, 22.10526316],
            ['2020-12-08', 74.0, 67.0, 139.0, 21.62162162],
            ['2020-12-09', 89.0, 82.0, 189.0, 21.34831461],
            ['2020-12-10', 81.0, 79.0, 154.0, 27.16049383],
            ['2020-12-11', 76.0, 70.0, 140.0, 25.0],
            ['2020-12-12', 70.0, 67.0, 134.0, 17.14285714],
            ['2020-12-13', 72.0, 70.0, 141.0, 1

In [23]:
service.spreadsheets().values().update(spreadsheetId=spreadsheetId, range=range_name, valueInputOption='USER_ENTERED', body=body).execute()

{'spreadsheetId': '1CmQqTVuzqDs5WX7fNAlB43AvTsoOfYqIE266impvCJI',
 'updatedRange': 'daily_metrics!A2:E31',
 'updatedRows': 30,
 'updatedColumns': 5,
 'updatedCells': 150}